<a href="https://colab.research.google.com/github/HmxaaK/Forecasting-Macroeconomic-Variables-through-Dynamic-Causal-Analysis/blob/main/Macroeconomics_Indicator_Forecasting_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from statsmodels.tsa import ardl
#Libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ardl import ARDL,ardl_select_order

#Dataset
data = pd.read_csv('GDP.csv', parse_dates=['observation_date'],
                   index_col='observation_date')
data = data.asfreq('QS')
#ADF test
def adf_test(data_set):
  non_stationary_vars = []
  for var in list(data.columns):
    adf = adfuller(data_set[var])
    if adf[1] > 0.10:
      non_stationary_vars.append(var)
  return non_stationary_vars
non_stationary_vars = adf_test(data)
print ('Non-Stationary Variable')
for var in non_stationary_vars:
  print (var)

#Log Differencing
log_GDP = np.log(data['GDPC1'])
Y = log_GDP - log_GDP.shift(1)
Y = Y.dropna()
log_Xs = np.log(data[non_stationary_vars[1:]])
log_diff_Xs = log_Xs - log_Xs.shift(1)
X = pd.concat([log_diff_Xs,data['UMCSENT']],axis=1).dropna()

model = ARDL(endog=Y,lags=0,exog=X,order=1,causal=True).fit()
print (model.summary())

forecasts = model.predict(start=0,end=(len(Y)))
forecasts.to_csv('forecast.csv')


Non-Stationary Variable
GDPC1
FEDFUND
DSPIC96
FGEXPND
EXPGSC1
                                ARDL Model Results                                
Dep. Variable:                      GDPC1   No. Observations:                  262
Model:             ARDL(0, 1, 1, 1, 1, 1)   Log Likelihood                 841.307
Method:                   Conditional MLE   S.D. of innovations              0.010
Date:                    Fri, 14 Nov 2025   AIC                          -1668.613
Time:                            15:12:10   BIC                          -1643.635
Sample:                        04-01-1960   HQIC                         -1658.574
                             - 04-01-2025                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0070      0.004     -1.599      0.111      -0.016       0.002
FEDFUND.L1    -0.0012      0.002     